In [1]:
# To get multiple outputs from one code cell (without using print()):
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image, Audio
import sys
from pathlib import Path
import pandas as pd
from pprint import pprint as pp

# For documenting the current environment:
def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))

# For enabling imports from current project code:
def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = F'Path already in sys.path: {newp}'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = F'Path added to sys.path: {newp}'
    print(msg)

# If this ipynb file is inside a folder, eg ./notebooks, 
# the project code is assumed to reside 1 level up:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), up=Path.cwd().name.startswith(nb_folder))

# For py modules/methods discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
       Example:
       >filter_dir(re) # lists the public methods of the re module.
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

# To create often-used subfolders:
def get_project_dirs(which=['data', 'images'],
                     use_parent=True):
    '''Create folder(s) named in `which` at the ipynb parent level.'''
    if use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    dir_lst = []    
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

# For adding colorfull divider in the nb:
def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)


def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:3em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')

# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2
# .................

Path added to sys.path: C:/Users/catch/Documents/GitHub/DU-event-transcript-demo/resources/EventManagement


In [2]:
import ipywidgets as ipw
from collections import OrderedDict

from manage import (EventMeta as Meta,
                    EventTranscription as TRX,
                    Workflow as FLO,
                    Utils as UTL)

main_readme = Meta.MAIN_README

In [5]:
FLO.show_du_logo_hdr()
FLO.DisplaySectionInfo(FLO.EventFunction.ADD).show_header_with_info()

In [149]:
# these 2 not yet used:
# save_actions_add = ['Update meta', 'Save meta', 'Create transcript', 'Update README table']
# show_actions = ['Show transcript', 'Show README']

# REMOVED: 'Validate' :: it's now a btn
sub_actions = [['Enter info', 'Save', 'Show'],
               ['Select event id', 'Modify table row',
                'Save', 'Show'],
               ['Select event id', 'Get transcript', 'Edit transcript',
                'Save', 'Show']
              ]
               
actions_d = OrderedDict([('Add an event', sub_actions[0]),
                         ('Modify an event', sub_actions[1]),
                         ('Edit a transcript',  sub_actions[2])
                        ])

# needed?: idx from action_d

# Input form components: ..........................................

def get_tabs(actions_d, tab_forms=None):
    tabs = ipw.Tab(layout=ipw.Layout(width='100%'))
    ks = list(actions_d.keys())
    if tab_forms is None:
        tabs.children = [ipw.HBox(description=k) for k in ks]
    else:
        tabs.children = tab_forms
    for i, k in enumerate(ks):
        # shortened bc tab width is fixed at 144px
        tabs.set_title(i, k.split()[0].upper())
    return tabs


def get_left_acc(acts_d, on_click_fn):
    """
    Creat Accordion with buttons children that have:
     - attributes: [name, idx] &
     - on_click function
    inside an .
    """
    sub_btns = []
    acc_items = [ipw.VBox(description=k) for k in list(acts_d.keys())]
    menu_acc = ipw.Accordion(children=acc_items,
                             selected_index=None,
                             layout=ipw.Layout(width='15%'))
    
    for i, (k, v) in enumerate(acts_d.items()):
        for j, s in enumerate(v):
            # only enable the 1st btn on the list:
            dsbl = False if j == 0 else True
            btn = ipw.Button(description=s,
                             disabled=dsbl)
            setattr(btn, 'name', s)
            setattr(btn, 'idx', j)
            setattr(btn, 'parent_idx', i)
            
            btn.on_click(on_click_fn)
            sub_btns.append(btn)
            
        menu_acc.children[i].children = sub_btns
        menu_acc.set_title(i, k.upper())
        sub_btns = []
    
    return menu_acc


def wgt_Show():
    #todo
    return

# ...................................................................

# Globals to receive instantiated TranscriptMeta & user entries     
TR = None
form_data_dict = None

validate_out = ipw.Output()

#@validate_out.capture()
def validate_new(entry_form, new_input_dict):
    global  TR
    global form_data_dict
    
    # metadata instance for new record:
    TR = Meta.TranscriptMeta()
    form_data_dict = FLO.validate_form_entries(entry_form,
                                               new_input_dict,
                                               TR)
    if form_data_dict is not None:
        # works?
        validate_out.value = 'Save: OK!'
        menu_acc.children[selected_index]
        #print('Validation: OK!')  #return needed?
    else:
        print('Validation: Fix & Try again.')
    return

@validate_out.capture()
def save_entry():
    try:
        TR.update_metadata(form_data_dict)
        TR.save_meta()
        TR.save_transcript_md()
        print('Save: OK!')
    except:
        print('Save: Something went wrong.')

@validate_out.capture()
def show_update():
    # add container to main_page?
    # or add a SHOW tab?
    if TR.transcript_md_path() == Meta.NA:
        print("No transcript file. Problem!")
        
    code = Markdown(filename=TR.transcript_md_path().data)
    tab_kids = page_tabs.children + [ipw.HTML(code)]
    page_tabs.children = tab_kids
    page_tabs.selected_index = 3
        
    
def menu_btn_onclick(wgt):
    if wgt.name == 'Save':
        save_entry()
    elif wgt.name == 'Show':
        show_update()
                    
    elif wgt.parent_idx == 0:
        if wgt.name == 'Enter info':
            page_tabs.children[wgt.parent_idx].children = [new_entry_group,
                                                           validate_vbx]
        elif wgt.name == 'Validate':
            validate_new(new_entry_group, new_input)
    elif wgt.parent_idx == 1:
        page_tabs.children[0].children = []
        if len(page_tabs.children) == 4:
            page_tabs.children[3].close()
        # temp, here should place idn sel box 1st &
        # set 'entry group' to disabled until idn selected.
        page_tabs.children[wgt.parent_idx].children = [new_entry_group,
                                                       validate_vbx]
    
        
# observe function
def menu_selection(change):
    tabs.selected_index = acco.selected_index

# ...............................................................    
# layouts:
main_layout = ipw.Layout(display='flex',
                         flex_flow='column',
                         align_items='stretch',
                         width='100%')

form_item_layout = ipw.Layout(display='flex',
                              flex_flow='row',
                              justify_content='space-between',
                              width='95%'
                             )

new_input = FLO.get_demo_input_dict()  # demo data items
children = [FLO.wgt_from_flds_dict(k, v) for k, v in new_input.items()]

new_entry_group = FLO.wgt_Acc(children) # wgt for each item

# OR: BTN=='VALIDATE' (+ OUT BOX):: IF MSG OK
#     'SAVE' IS ENABLED
# REUSE OUT BOX FOR SSAVE OUTCOME:: IF MSG OK
#      'SHOW' IS ENABLED.
# ==> REMOVE 'Validate' FROM ACC MENU


validate_btn = ipw.Button(description='Validate',
                      button_style='success',
                      icon='check', disabled=True)
# moved need to occur before deroator
#validate_out = ipw.Output()
validate_vbx = ipw.VBox([validate_btn, validate_out])

# Get page components:
page_tabs = get_tabs(actions_d)
menu_acc = get_left_acc(actions_d, menu_btn_onclick)

# todo if OK in text:
#  menu_acc[selected_index].children[0].disabled = True
#  menu_acc[selected_index].children[1].disabled = False
#menu_acc.observe(validate_out, 'text')

# Link accodrion selection index with tab index:
menu_acc.observe(menu_selection, names='value')
dl0 = ipw.dlink((menu_acc, 'selected_index'), (page_tabs, 'selected_index'))

# Main container:
main_page = ipw.VBox([ipw.HBox([menu_acc, page_tabs],
                               layout=form_item_layout)],
                     layout=main_layout)

NameError: name 'validate_out' is not defined

In [145]:
main_page

---
---

## Step 1: instantiate new TranscriptMeta class

In [153]:
tr = Meta.TranscriptMeta()

## Step 2: build the entry group to supply each of the main keys, with exception for video: video_url => split

## Step 3: Save btn steps:
### Step 3.1 Retrieve data from form

### Step 3.2: update metadata & save to json:

### Step 4: create new transcript md file

### Step 5: Show created md file

In [204]:
Markdown('---')
Markdown(filename=tr.transcript_md_path())
Markdown('---')

---

Cat Chenal: My Presentation Title
=================================

Key Links
---------

-   Transcript:
    <https://github.com/data-umbrella/event-transcripts/blob/main/2020/18-cat-chenal-bar-foo-foo.md>
-   Meetup Event: N.A.
-   Video: <https://youtu.be/MHAjCcBfT_A>
-   Slides: N.A.
-   GitHub repo: <https://github.com/CatChenal>
-   Jupyter Notebook: N.A.
-   Transcriber: N.A.

Other References
----------------

-   Binder: url
-   Paper: url or citation.

Wiki of Note
------------

-   Wiki: This is an excellent [wiki on audio
    transcription](http://en.wikipedia.org/wiki/Main_Page)

Video
-----

<div markdown="1" width="25%">

[![My Presentation
Title](http://img.youtube.com/vi/MHAjCcBfT_A/0.jpg)](http://www.youtube.com/watch?feature=player_embedded&v=MHAjCcBfT_A)

</div>

Transcript
----------

N.A.


---

In [205]:
raw_md = Markdown(filename=tr.transcript_md_path()).data
pp(raw_md)

('Cat Chenal: My Presentation Title\n'
 '=================================\n'
 '\n'
 'Key Links\n'
 '---------\n'
 '\n'
 '-   Transcript:\n'
 '    '
 '<https://github.com/data-umbrella/event-transcripts/blob/main/2020/18-cat-chenal-bar-foo-foo.md>\n'
 '-   Meetup Event: N.A.\n'
 '-   Video: <https://youtu.be/MHAjCcBfT_A>\n'
 '-   Slides: N.A.\n'
 '-   GitHub repo: <https://github.com/CatChenal>\n'
 '-   Jupyter Notebook: N.A.\n'
 '-   Transcriber: N.A.\n'
 '\n'
 'Other References\n'
 '----------------\n'
 '\n'
 '-   Binder: url\n'
 '-   Paper: url or citation.\n'
 '\n'
 'Wiki of Note\n'
 '------------\n'
 '\n'
 '-   Wiki: This is an excellent [wiki on audio\n'
 '    transcription](http://en.wikipedia.org/wiki/Main_Page)\n'
 '\n'
 'Video\n'
 '-----\n'
 '\n'
 '<div markdown="1" width="25%">\n'
 '\n'
 '[![My Presentation\n'
 'Title](http://img.youtube.com/vi/MHAjCcBfT_A/0.jpg)](http://www.youtube.com/watch?feature=player_embedded&v=MHAjCcBfT_A)\n'
 '\n'
 '</div>\n'
 '\n'
 'Transcript\n'
 

## After displaying transcript_md, ask Update README table?: